In [ ]:
import tensorflow as tf
import pandas as pd
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
# Load and preprocess dataset
def load_annotations(csv_file, image_dir):
    data = pd.read_csv(csv_file)
    data['image_path'] = data['filename'].apply(lambda x: os.path.join(image_dir, x))
    return data

In [ ]:
# Prepare TensorFlow dataset
def preprocess_data(df):
    images = []
    labels = []
    bboxes = []

    for idx, row in df.iterrows():
        image = tf.image.decode_jpeg(tf.io.read_file(row['image_path']))
        image = tf.image.resize(image, (224, 224))
        images.append(image)

        labels.append(row['class'])
        bbox = [
            row['xmin'] / row['width'],
            row['ymin'] / row['height'],
            row['xmax'] / row['width'],
            row['ymax'] / row['height'],
        ]
        bboxes.append(bbox)

    images = tf.stack(images)
    labels = tf.convert_to_tensor(labels)
    bboxes = tf.convert_to_tensor(bboxes)

    return images, labels, bboxes

In [ ]:
# Build Fast R-CNN Model
def build_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = Flatten()(base_model.output)
    classification_output = Dense(num_classes, activation='softmax', name='classification')(x)
    bbox_output = Dense(4, activation='sigmoid', name='bbox_regression')(x)

    model = Model(inputs=base_model.input, outputs=[classification_output, bbox_output])
    return model

In [ ]:
# Training Setup
def train_model(train_data, val_data, num_classes, epochs=10):
    train_images, train_labels, train_bboxes = train_data
    val_images, val_labels, val_bboxes = val_data

    model = build_model(num_classes)

    model.compile(
        optimizer='adam',
        loss={
            'classification': 'sparse_categorical_crossentropy',
            'bbox_regression': 'mse',
        },
        metrics={
            'classification': 'accuracy',
            'bbox_regression': 'mae',
        },
    )

    model.fit(
        x=train_images,
        y={'classification': train_labels, 'bbox_regression': train_bboxes},
        validation_data=(
            val_images,
            {'classification': val_labels, 'bbox_regression': val_bboxes},
        ),
        epochs=epochs,
        batch_size=32,
    )

    return model

In [ ]:
# Paths to your dataset
train_csv = "path_to_train_annotations.csv"
val_csv = "path_to_val_annotations.csv"
test_csv = "path_to_test_annotations.csv"
train_image_dir = "path_to_train_images_directory"
val_image_dir = "path_to_val_images_directory"
test_image_dir = "path_to_test_images_directory"

In [ ]:
# Load datasets
train_df = load_annotations(train_csv, train_image_dir)
val_df = load_annotations(val_csv, val_image_dir)
test_df = load_annotations(test_csv, test_image_dir)

In [ ]:
# Preprocess datasets
train_data = preprocess_data(train_df)
val_data = preprocess_data(val_df)
test_data = preprocess_data(test_df)


In [ ]:
# Number of classes (you can modify based on your dataset)
num_classes = train_df['class'].nunique()


In [ ]:
# Train the model
model = train_model(train_data, val_data, num_classes, epochs=5)

In [ ]:
# Save the trained model
model.save("fast_rcnn_model.h5")